In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pickle

import csv
import os

from gpu import set_gpu

import tensorflow as tf
from latent.ml.dl_subclass import MLP, MLPbeta, CNN, MLPprop, get_train
from latent.ml.dl_manual import nn_pass, conv, bn, dense
import latent.session_subclass as session
import latent.utils.data_utils as prd
from latent.ml.lda import train_lda, eval_lda

set_gpu()

Using TensorFlow backend.


Num GPUs Available:  1


In [2]:
data, params = prd.load_caps_train('traindata.mat')

In [5]:
ep = 30
n_dof = np.max(params[:,0])
    
# Train NNs
mlp = MLP(n_class=n_dof)
mlp_beta = MLPbeta(n_class=n_dof)
mlp_prop = MLPprop(n_class=n_dof)
cnn1 = CNN(n_class=n_dof,c1=16,c2=8)
cnn2 = CNN(n_class=n_dof,c1=32,c2=32)
cnn = CNN(n_class=n_dof,c1=8,c2=8)

optimizer = tf.keras.optimizers.Adam()
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')
train_prop_accuracy = tf.keras.metrics.MeanSquaredError(name='train_prop_accuracy')

# Train neural networks
models = [mlp, mlp_beta, cnn]
models = [cnn2, mlp_prop]

trainmlp, traincnn, y_train, x_train_mlp, x_train_cnn, x_train_lda, y_train_lda, x_train_aug, emg_scale, scaler, x_min, x_max, prop = prd.prep_train_caps(data, params)

for model in models:
    if isinstance(model,CNN):
        ds = traincnn
    else:
        ds = trainmlp
 
    prop_b = isinstance(model, MLPprop) or isinstance(model, CNN)
 
    train_mod = get_train(prop = prop_b)

    for epoch in range(ep):
        # Reset the metrics at the start of the next epoch
        train_loss.reset_states()
        train_accuracy.reset_states()
        train_prop_accuracy.reset_states()

        for x, y, y2 in ds:
            if prop_b:
                train_mod(x, y, model, optimizer, train_loss, train_accuracy, train_prop_accuracy, y2)
            else:
                train_mod(x, y, model, optimizer, train_loss, train_accuracy)

        if epoch == 0 or epoch == ep-1:
            print(
                f'Epoch {epoch + 1}, '
                f'Loss: {train_loss.result():.2f}, '
                f'Accuracy: {train_accuracy.result() * 100:.2f}, '
                f'MSE: {train_prop_accuracy.result():.2f} '
            )
            
    del train_mod

# Train aligned LDA
# y_train_aug = np.argmax(y_train, axis=1)[...,np.newaxis]

# mlp_enc = mlp.get_layer(name='enc')
# w_mlp, c_mlp,_, _, _ = train_lda(mlp_enc(x_train_mlp).numpy(),y_train_aug)

# mlpbeta_enc = mlp_beta.get_layer(name='enc')
# w_mlpbeta, c_mlpbeta,_, _, _ = train_lda(mlpbeta_enc(x_train_mlp).numpy(),y_train_aug)

# cnn_enc = cnn.get_layer(name='enc')
# temp = cnn_enc(x_train_cnn[:x_train_cnn.shape[0]//2,...]).numpy()
# temp2 = np.vstack((temp,cnn_enc(x_train_cnn[x_train_cnn.shape[0]//2:,...]).numpy()))
# w_cnn, c_cnn,_, _, _ = train_lda(temp2,y_train_aug)

# # Train LDA
# w,c, _, _, _ = train_lda(x_train_lda,y_train_lda)
# w_aug,c_aug, _, _, _ = train_lda(x_train_aug,y_train_aug)

mlp_w = mlp.get_weights()
mlpb_w = mlp_beta.get_weights()
cnn_w = cnn2.get_weights()
mlpp_w = mlp_prop.get_weights()

NameError: name 'cn2' is not defined

In [4]:
# Save neural network architectures into csv
cnn_arch = ['CONV1','CBN1','CONV2','CBN2','FLAT','CDEN','CBN3','CLIN','CSOFTMAX','CPROP']
mlp_arch = ['RELU1','BN1','RELU2','BN2','RELU3','BN3','DEN','BN4','SOFTMAX','PROP']
arch = cnn_arch
w_in = cnn_w
folder = 'weights_cnn'
nn = 'C'
path = '//192.168.20.2/config/modes/caps/upper_limb/dl_ctrl/DATA/'
if not os.path.isdir(path):
    path =  ''
    print('no embedded')

i = 0
for l in arch:
    w_layer = []
    if 'BN' in l:
        w_layer = np.vstack((np.vstack((np.vstack((w_in[i],w_in[i+1])),w_in[i+2])),w_in[i+3]))
        i += 4
    elif 'CONV' in l:
        w_layer = np.vstack((w_in[i].reshape((-1,w_in[i].shape[-1])),w_in[i+1]))

        with open(path + folder + '/'  + l + '_shape.csv', 'w', newline='') as f:
            writer = csv.writer(f)
            # write multiple rows
            writer.writerow(w_in[i].shape)
        i += 2
    elif 'FLAT' not in l:
        w_layer = np.vstack((w_in[i],w_in[i+1]))
        i += 2      
    
    if 'FLAT' not in l:
        # Save to embedded
        with open(path + folder + '/'  + l + '.csv', 'w', newline='') as f:
            writer = csv.writer(f)
            # write multiple rows
            writer.writerows(w_layer)

fill = len(emg_scale) - len(x_min)
temp = np.hstack((np.vstack((x_min[...,np.newaxis],np.zeros((fill,1)))), np.vstack((x_max[...,np.newaxis],np.zeros((fill,1))))))
scales = np.hstack((emg_scale,temp))

# Save to current folder
with open(path + folder + '/' + nn + 'ARCH.csv', 'w', newline='') as f:
    writer = csv.writer(f,delimiter=',')
    # write multiple rows
    for i in arch:
        writer.writerow([i])

with open(path + folder + '/' + nn + 'scales.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    # write multiple rows
    writer.writerows(scales)